In [81]:
from datetime import datetime
# from dateutil.relativedelta import relativedelta
import math
from marketdatalib import map_scrip_to_yfin_ticker, get_nifty_index_data, calculate_beta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from scipy.optimize import minimize

In [82]:
# Number of year of historic data to consider
YEARS_OF_HISTORY = 5

# The risk free return rate. This is often the return on a 10 year government bond.
# At present, the 10 year government bond rate is 6.84%. We are using 7.5% for the sake of simplicity which is the rate of a fixed deposit.
RISK_FREE_RETURN = 0.072
EXPECTED_MARKET_RETURN = 0.18
ACCEPTABLE_VOLATILITY = 0.15


INDIAN_ETFS = ['NIFTY 100', 'NIFTY 500', 'NIFTY MIDCAP 150', 'NIFTY NEXT 50']
INDIAN_ETFS

['NIFTY 100', 'NIFTY 500', 'NIFTY MIDCAP 150', 'NIFTY NEXT 50']

In [90]:
master_data = pd.DataFrame()

for ind in INDIAN_ETFS:
    index_data = get_nifty_index_data(index_name=ind, number_of_years=YEARS_OF_HISTORY)['Close']
    index_data = pd.Series(index_data)
    index_data = index_data.to_frame(name=ind)
    master_data = master_data.combine_first(index_data)

# For every column in the master_data, add a new columns with name <column_name>_returns containting percentate returns compared to previous day
for column in master_data:
    master_data[column + '_Return'] = master_data[column].pct_change() + 1

master_data = master_data.dropna()

master_data

,NIFTY 100,NIFTY 500,NIFTY MIDCAP 150,NIFTY NEXT 50,NIFTY 100_Return,NIFTY 500_Return,NIFTY MIDCAP 150_Return,NIFTY NEXT 50_Return
Date,,,,,,,,
2020-01-07,12154.95,9805.40,6303.85,28101.20,1.005322,1.005950,1.007198,1.007396
2020-01-08,12131.45,9792.20,6315.40,28113.55,0.998067,0.998654,1.001832,1.000439
2020-01-09,12319.70,9943.90,6403.55,28484.30,1.015518,1.015492,1.013958,1.013188
2020-01-10,12359.70,9978.00,6422.95,28556.05,1.003247,1.003429,1.003030,1.002519
2020-01-13,12436.45,10042.50,6465.75,28783.55,1.006210,1.006464,1.006664,1.007967
...,...,...,...,...,...,...,...,...
2024-12-30,24500.75,22357.15,21109.25,68063.80,0.994672,0.996077,1.001775,0.992804
2024-12-31,24495.55,22375.40,21141.20,67988.35,0.999788,1.000816,1.001514,0.998891
2025-01-01,24595.70,22481.80,21246.50,68247.95,1.004088,1.004755,1.004981,1.003818


In [91]:
def expected_return(market_return, risk_free_return, beta, weight = 1):
    return risk_free_return + weight * beta * (market_return - risk_free_return)

Let us assume that NIFTY 500 represents the total market for the sake of brevity. It represnts the universe of stocks that are investable in Indian equity market

In [92]:
market_returns = master_data['NIFTY 500_Return']

R_m = market_returns.prod() - 1
Sigma_m = market_returns.std()

print(f"Market Return: {R_m}")
print(f"Market Volatility: {Sigma_m}")


Market Return: 1.329662268912743
Market Volatility: 0.011409597317991297


To start with, let us start with a single security portfolio. The portfolio is constructed with NIFTY 50 and NIFTY PHARMA index.

In [93]:
portfolio_indices = ['NIFTY 100', 'NIFTY 500', 'NIFTY MIDCAP 150', 'NIFTY NEXT 50']

In [95]:
returns_matrix = pd.DataFrame(index=portfolio_indices, columns=['Return', 'Beta', 'Volatility', 'ExpectedReturn', 'Sharpe Ratio'])

for index in portfolio_indices:
    index_returns = master_data[index + '_Return'].prod()
    returns_matrix.loc[index, 'Return'] = index_returns
    returns_matrix.loc[index, 'Beta'] = calculate_beta(master_data[index + '_Return'], market_returns)
    returns_matrix.loc[index, 'Volatility'] = master_data[index + '_Return'].std()
    returns_matrix.loc[index, 'ExpectedReturn'] = RISK_FREE_RETURN + returns_matrix.loc[index, 'Beta'] * (R_m - RISK_FREE_RETURN)
    returns_matrix.loc[index, 'Sharpe Ratio'] = (index_returns - RISK_FREE_RETURN) / returns_matrix.loc[index, 'Volatility']

# variannce_i = master_data['NIFTY 100_Return'].var()
# Sigma_i = np.sqrt(variannce_i)
# corr_i_m = master_data['NIFTY 100_Return'].corr(market_returns)
# beta_i = corr_i_m * Sigma_i / Sigma_m
# Sigma_i, beta_i
returns_matrix

,Return,Beta,Volatility,ExpectedReturn,Sharpe Ratio
NIFTY 100,2.057785,1.007893,0.011575,1.339589,171.561698
NIFTY 500,2.329662,1.000767,0.01141,1.330627,197.873966
NIFTY MIDCAP 150,3.422269,0.956851,0.011803,1.275396,283.849467
NIFTY NEXT 50,2.48047,0.975925,0.012127,1.299384,198.608529


In [88]:
weights = np.linspace(0, 100, 100)

returns = [expected_return(market_return=R_m, risk_free_return=RISK_FREE_RETURN, beta=beta_i, weight=weight) for weight in weights]
volatilities = [Sigma_i * weight * 100 for weight in weights]


NameError: name 'beta_i' is not defined

In [ ]:
ax = plt.figure(figsize=(12, 12))

plt.scatter(volatilities, returns, c=volatilities, cmap='viridis')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
master_data['NIFTY 100_Return'].prod() - 1

In [ ]:
volatilities